# King County , WA House Price Analysis and Prediction

## Business Problem

You are hired by a real estate agency to analize and predict the price of a house in King County, WA area based on a defferent house features.
- This analysis is focused on the house prices for the years 2014 and 2015 housing sales in King County, WA.
- Multiple linear regression models are used to achieve this goal.

## Data 
- King County housing 2014-2015 data is collected from [Kaggle](https://www.kaggle.com/code/mosesaborisade/moses-house-sales-in-king-county-usa/data). 

Data Dictionary:

- id: A unique sale id relating to a house sale
- date: Date of house sale
- price: The price which the house sold for
- bedrooms: No of bedrooms house has
- bathrooms: No of bathrooms house has
- sqft_living: Square footage of the house 
- sqft_lot: Square footage of the lot 
- floors: No of floors house has
- waterfront: Whether the house is near/on waterfront. Originally contained ‘YES’ or ‘NO’, converted to 0 or 1 for comparative purposes
- view: Whether the house has a view and whether it’s fair, average, good, or excellent. Converted to numberical (0-4) for comparative purposes
- condition: overall condition of the house: Poor, Fair, Average, Good, Very Good
- grade: Numerical grading for house
- sqft_above: How much of the houses square footage is above ground
- sqft_basement: How much of the square footage is in the basement
- yr_built: Year the house was built
- yr_renovated: Year the house was renovated, if applicable
- zipcode: House zipcode
- lat: House’s latitude coordinate
- long: House’s longitude coordinate
- sqft_living15: Average size of living space for the closest 15 houses
- sqft_lot15: Average size of lot for the closest 15 houses


In [2]:
import pandas as pd


In [5]:
df =pd.read_csv('Data/kc_house_data.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  